<a href="https://colab.research.google.com/github/doantronghieu/DEEP-LEARNING/blob/main/CSR_TF_Dev_Pro_Cert/C1/C1_W2_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 2: Implementing Callbacks in TensorFlow using the MNIST Dataset

In the course you learned how to do classification using Fashion MNIST, a data set containing items of clothing. There's another, similar dataset called MNIST which has items of handwriting -- the digits 0 through 9.

Write an MNIST classifier that trains to 99% accuracy or above, and does it without a fixed number of epochs -- i.e. you should stop training once you reach that level of accuracy. In the lecture you saw how this was done for the loss but here you will be using accuracy instead.

Some notes:
1. Given the architecture of the net, it should succeed in less than 10 epochs.
2. When it reaches 99% or greater it should print out the string "Reached 99% accuracy so cancelling training!" and stop training.
3. If you add any additional variables, make sure you use the same names as the ones used in the class. This is important for the function signatures (the parameters and names) of the callbacks.

In [1]:
import os
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as layers
import tensorflow.keras.optimizers as optimizers
import tensorflow.nn as nn

Begin by loading the data. A couple of things to notice:

- The file `mnist.npz` is already included in the current workspace under the `data` directory. By default the `load_data` from Keras accepts a path relative to `~/.keras/datasets` but in this case it is stored somewhere else, as a result of this, you need to specify the full path.

- `load_data` returns the train and test sets in the form of the tuples `(x_train, y_train), (x_test, y_test)` but in this exercise you will be needing only the train set so you can ignore the second tuple.

In [20]:
import zipfile
!wget https://github.com/doantronghieu/DEEP-LEARNING/blob/main/CSR_TF_Dev_Pro_Cert/C1/data.zip?raw=true
zip_ref = zipfile.ZipFile("data.zip?raw=true", "r")
zip_ref.extractall()
zip_ref.close()

--2022-04-13 23:53:20--  https://github.com/doantronghieu/DEEP-LEARNING/blob/main/CSR_TF_Dev_Pro_Cert/C1/data.zip?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/doantronghieu/DEEP-LEARNING/raw/main/CSR_TF_Dev_Pro_Cert/C1/data.zip [following]
--2022-04-13 23:53:20--  https://github.com/doantronghieu/DEEP-LEARNING/raw/main/CSR_TF_Dev_Pro_Cert/C1/data.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/doantronghieu/DEEP-LEARNING/main/CSR_TF_Dev_Pro_Cert/C1/data.zip [following]
--2022-04-13 23:53:20--  https://raw.githubusercontent.com/doantronghieu/DEEP-LEARNING/main/CSR_TF_Dev_Pro_Cert/C1/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.gi

In [23]:
# Get the current working directory
current_dir = os.getcwd()

# Append data/mnist.npz to the previous path to get the full path
data_path = os.path.join(current_dir, 'data/mnist.npz')

# Discard test set
(x_train, y_train), _ = tfk.datasets.mnist.load_data(path=data_path)

# Normalize the pixel values
x_train = x_train / 255.0

print(current_dir)
print(data_path)

/content
/content/data/mnist.npz


Now take a look at the shape of the training data:

In [24]:
data_shape = x_train.shape

print(f'There are {data_shape[0]} examples with shape ({data_shape[1]}, {data_shape[2]})')

There are 60000 examples with shape (28, 28)


Now it is time to create your own custom callback. For this complete the `myCallback` class and the `on_epoch_end` method in the cell below. If you need some guidance on how to proceed, check out this [link](https://www.tensorflow.org/guide/keras/custom_callback).

In [25]:
# Inherit from the correct class
class myCallback(tfk.callbacks.Callback):
    # Define the function signature
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') is not None and logs.get('accuracy') > 0.99):
            print('\nReached 99% accuracy so cancelling training!')

            # Stop training once the above condition is met
            self.model.stop_training = True

Now that you have defined your callback it is time to complete the `train_mnist` function below:

In [26]:
def train_mnist(x_train, y_train):
    
    # Instantiate the callback class
    callbacks = myCallback()

    # Define the model, it should have 3 layers:
    # - A Flatten layer that receives inputs with the same shape as the images
    # - A Dense layer with 512 units and ReLU activation function
    # - A Dense layer with 10 units and softmax activation function
    model = tfk.models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(512, activation=nn.relu),
        layers.Dense(10, activation=nn.softmax)                                   
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Fit the model, add the callbacks, save the training accuracy
    history = model.fit(x_train, y_train, epochs=10, callbacks=[callbacks])

    return history
    

Call the `train_mnist` passing in the appropiate parameters to get the training history:

In [27]:
hist = train_mnist(x_train, y_train)

Epoch 1/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2011 - accuracy: 0.9413
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0808 - accuracy: 0.9754
Epoch 3/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0530 - accuracy: 0.9835
Epoch 4/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0362 - accuracy: 0.9884
Epoch 5/10
1873/1875 [============================>.] - ETA: 0s - loss: 0.0270 - accuracy: 0.9913
Reached 99% accuracy so cancelling training!
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0270 - accuracy: 0.9913


If you see the message `Reached 99% accuracy so cancelling training!` printed out after less than 10 epochs it means your callback worked as expected. 

**Congratulations on finishing this week's assignment!**

You have successfully implemented a callback that gives you more control over the training loop for your model. Nice job!

**Keep it up!**